# 1D Convolution on FPGA

## 1. Set-up

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [3]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/ece5545’: File exists
/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-wenfyan' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-wenfyan
M	src/ops.py
M	src/ops_fpga.py
Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [5]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb   4-gemm_gpu.ipynb	    README.md
2-conv1d_gpu.ipynb   5-conv2d_dw_gpu.ipynb  src
3-conv1d_fpga.ipynb  leaderboard_id.txt     tests


## 2. Install TVM and VTA Simulator

In [6]:
! gsutil cp "gs://tvm-fcrc-binaries-7f775516ff9dfab922c304049f294cec/tvm.tar.gz" /tmp/tvm.tar.gz
! mkdir -p /tvm
! tar -xf /tmp/tvm.tar.gz --strip-components=4 --directory /tvm
! ls -la /tvm
! bash /tvm/package.sh

Copying gs://tvm-fcrc-binaries-7f775516ff9dfab922c304049f294cec/tvm.tar.gz...
\ [1 files][114.5 MiB/114.5 MiB]                                                
Operation completed over 1 objects/114.5 MiB.                                    
total 164
drwxr-xr-x 21 root root  4096 Apr 12 04:07 .
drwxr-xr-x  1 root root  4096 Apr 12 00:10 ..
drwx------  8 root root  4096 Jun 11  2019 3rdparty
drwx------ 12 root root  4096 Jun 11  2019 apps
drwx------  3 root root  4096 Jun 12  2019 build
drwx------  4 root root  4096 Jun 11  2019 cmake
-rw-------  1 root root 10406 Jun 11  2019 CMakeLists.txt
drwx------  6 root root  4096 Jun 11  2019 conda
-rw-------  1 root root  5673 Jun 11  2019 CONTRIBUTORS.md
drwx------  3 root root  4096 Jun 11  2019 docker
drwx------ 11 root root  4096 Jun 11  2019 docs
drwx------  4 root root  4096 Jun 11  2019 golang
drwx------  3 root root  4096 Jun 11  2019 include
-rw-------  1 root root 10542 Jun 11  2019 Jenkinsfile
drwx------  6 root root  4096 Jun 11  20

In [7]:
!pip install numpy==1.24.3

In [36]:

!chmod +x /content/cuda_10.0.130_410.48_linux.run

!/content/cuda_10.0.130_410.48_linux.run \
  --silent \
  --toolkit \
  --toolkitpath=/content/gdrive/MyDrive/ece5545/cuda-10.0 \
  --override \
  --no-opengl-libs

In [9]:

!chmod +x cuda_10.0.130_410.48_linux.run

!mkdir -p /content/cuda-10.0

!./cuda_10.0.130_410.48_linux.run \
  --silent \
  --toolkit \
  --toolkitpath=/content/cuda-10.0 \
  --override \
  --no-opengl-libs

In [9]:
import os
CUDA_INSTALL_PATH = "/content/gdrive/MyDrive/ece5545/cuda-10.0"
os.environ['PATH'] = f"{CUDA_INSTALL_PATH}/bin:" + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = f"{CUDA_INSTALL_PATH}/lib64:" + os.environ.get('LD_LIBRARY_PATH', '')

In [10]:
LLVM_INSTALL_PATH = '/content/gdrive/MyDrive/ece5545/llvm-6.0'

os.environ['PATH'] = f"{LLVM_INSTALL_PATH}/bin:" + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = f"{LLVM_INSTALL_PATH}/lib:" + os.environ.get('LD_LIBRARY_PATH', '')

!which clang
!ls {LLVM_INSTALL_PATH}/lib/libLLVM-6.0.so.1

/usr/bin/clang
/content/gdrive/MyDrive/ece5545/llvm-6.0/lib/libLLVM-6.0.so.1


In [14]:

%cd /content/gdrive/MyDrive/ece5545/llvm-6.0/lib

!ln -s libLLVM-6.0.0.so libLLVM-6.0.so.1

!ls -l libLLVM-6.0.so.1

/content/gdrive/MyDrive/ece5545/llvm-6.0/lib
ln: failed to create symbolic link 'libLLVM-6.0.so.1': File exists
lrw------- 1 root root 16 Apr 12 04:05 libLLVM-6.0.so.1 -> libLLVM-6.0.0.so


In [27]:
!mkdir -p "/content/gdrive/MyDrive/ece5545/cuda-10.0"

In [8]:
!wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda_10.0.130_410.48_linux -O cuda_10.0.130_410.48_linux.run

--2025-04-11 22:19:36--  https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda_10.0.130_410.48_linux
Resolving developer.nvidia.com (developer.nvidia.com)... 23.50.232.200, 104.88.70.104
Connecting to developer.nvidia.com (developer.nvidia.com)|23.50.232.200|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://developer.nvidia.com/downloads/compute/cuda/10.0/prod/local_installers/cuda_10.0.130_410.48_linux [following]
--2025-04-11 22:19:37--  https://developer.nvidia.com/downloads/compute/cuda/10.0/prod/local_installers/cuda_10.0.130_410.48_linux
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://developer.download.nvidia.com/compute/cuda/10.0/secure/Prod/local_installers/cuda_10.0.130_410.48_linux.run?__token__=exp=1744410577~hmac=57fe4a9f47dc88aa89f3d2df5833857049cf8ef79fd6e4906cc6f1b899ed993a [following]
--2025-04-11 22:19:37--  http

In [14]:
!wget https://releases.llvm.org/6.0.0/clang+llvm-6.0.0-x86_64-linux-gnu-ubuntu-16.04.tar.xz
!tar -xf clang+llvm-6.0.0-x86_64-linux-gnu-ubuntu-16.04.tar.xz
!mv clang+llvm-6.0.0-x86_64-linux-gnu-ubuntu-16.04 /content/llvm-6.0

--2025-04-11 22:38:34--  https://releases.llvm.org/6.0.0/clang+llvm-6.0.0-x86_64-linux-gnu-ubuntu-16.04.tar.xz
Resolving releases.llvm.org (releases.llvm.org)... 199.232.194.49, 199.232.198.49, 2a04:4e42:4c::561, ...
Connecting to releases.llvm.org (releases.llvm.org)|199.232.194.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 292373416 (279M) [application/octet-stream]
Saving to: ‘clang+llvm-6.0.0-x86_64-linux-gnu-ubuntu-16.04.tar.xz.4’

clang+llvm-6.0.0-x8 100%[===================>] 278.83M  23.4MB/s    in 13s     

2025-04-11 22:38:48 (21.4 MB/s) - ‘clang+llvm-6.0.0-x86_64-linux-gnu-ubuntu-16.04.tar.xz.4’ saved [292373416/292373416]



In [19]:
!wget https://releases.llvm.org/6.0.0/llvm-6.0.0.src.tar.xz
!tar -xf llvm-6.0.0.src.tar.xz

--2025-04-12 00:55:54--  https://releases.llvm.org/6.0.0/llvm-6.0.0.src.tar.xz
Resolving releases.llvm.org (releases.llvm.org)... 199.232.194.49, 199.232.198.49, 2a04:4e42:4c::561, ...
Connecting to releases.llvm.org (releases.llvm.org)|199.232.194.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25296048 (24M) [application/octet-stream]
Saving to: ‘llvm-6.0.0.src.tar.xz’

llvm-6.0.0.src.tar. 100%[===================>]  24.12M  11.7MB/s    in 2.1s    

2025-04-12 00:55:57 (11.7 MB/s) - ‘llvm-6.0.0.src.tar.xz’ saved [25296048/25296048]



In [21]:
!mkdir llvm-build && cd llvm-build
!cmake ../llvm-6.0.0.src \
  -DCMAKE_INSTALL_PREFIX=/content/gdrive/MyDrive/ece5545/llvm-6.0 \
  -DCMAKE_BUILD_TYPE=Release \
  -DLLVM_BUILD_LLVM_DYLIB=ON \
  -DLLVM_LINK_LLVM_DYLIB=ON \
  -DLLVM_ENABLE_RTTI=ON

CMake Deprecation Warning at CMakeLists.txt:3 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


CMake Deprecation Warning at CMakeLists.txt:14 (cmake_policy):
  The OLD behavior for policy CMP0051 will be removed from a future version
  of CMake.

  The cmake-policies(7) manual explains that the OLD behaviors of all
  policies are deprecated and that a policy should be set to OLD only under
  specific short-term circumstances.  Projects should be ported to the NEW
  behavior and not rely on setting a policy to OLD.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Detecting C compiler ABI info


In [22]:
!make -j4
!make install

流式输出内容被截断，只能显示最后 5000 行内容。
[ 88%] Building CXX object tools/llvm-profdata/CMakeFiles/llvm-profdata.dir/llvm-profdata.cpp.o
In file included from /content/gdrive/MyDrive/ece5545/llvm-6.0.0.src/include/llvm/ADT/STLExtras.h:21,
                 from /content/gdrive/MyDrive/ece5545/llvm-6.0.0.src/tools/lto/lto.cpp:16:
/content/gdrive/MyDrive/ece5545/llvm-6.0.0.src/include/llvm/ADT/SmallVector.h: In instantiation of ‘void llvm::SmallVectorTemplateBase<T, true>::push_back(const T&) [with T = std::pair<void*, long unsigned int>]’:
/content/gdrive/MyDrive/ece5545/llvm-6.0.0.src/include/llvm/Support/Allocator.h:247:33:   required from ‘void* llvm::BumpPtrAllocatorImpl<AllocatorT, SlabSize, SizeThreshold>::Allocate(size_t, size_t) [with AllocatorT = llvm::MallocAllocator; long unsigned int SlabSize = 4096; long unsigned int SizeThreshold = 4096; size_t = long unsigned int]’
/content/gdrive/MyDrive/ece5545/llvm-6.0.0.src/include/llvm/Support/YAMLParser.h:130:26:   required from here
/content/gdri

In [35]:
!sudo rm -f /etc/apt/sources.list.d/sbt*.list

!echo "deb https://repo.scala-sbt.org/scalasbt/debian all main" | sudo tee /etc/apt/sources.list.d/sbt.list
!curl -sL "https://keyserver.ubuntu.com/pks/lookup?op=get&search=0x2EE0EA64E40A89B84B2DF73499E82A75642AC823" | sudo apt-key add

!sudo apt update
!sudo apt install sbt

deb https://repo.scala-sbt.org/scalasbt/debian all main
OK
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:5 https://scala.jfrog.io/artifactory/debian all InRelease [4,417 B]
Get:12 https://scala.jfrog.io/artifactory/debian all/main amd64 Packages [3,599 B]
Fetched 137 kB in 2s (66.3 kB/s)
Reading package lists... Done
Bu

In [17]:
import ctypes

ctypes.CDLL('/content/gdrive/MyDrive/ece5545/cuda-10.0/lib64/libnvrtc.so.10.0')

ctypes.CDLL('/content/gdrive/MyDrive/ece5545/llvm-6.0/lib/libLLVM-6.0.so.1')

print("environment check")

environment check


## 3. Implement `make_conv1d_fpga_*` functions in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it on a simulator of FPGA.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_gpu_scheduler` takes $m$ and $n$, which are the size of the two 1D input array.
You should return both the TVM scheduler and the TVM opterator for
1. Input $x$
2. Input $y$
3. Output $out$
Other than these, you should also return the environment and the remote object as already provided in the template code.
These objects will be useful for the next function `make_conv1d_fpga_function`.

The `make_conv1d_fpga_function` will provide means to use the numpy arrays to run in the FPGA simulator.
Specifically, it will take the information output by the `make_conv1d_gpu_scheduler` and return a function `f` which takes two numpy array and compute the output numpy array.

NOTE: the computation inside `f` must use the scheduler output from `make_conv1d_fpga_scheduler`.
You shouldn't replace the computation with a numpy or torch library call.

In [12]:
# Add TVM to the Python path.
import sys
sys.path.append('/tvm/python')
sys.path.append('/tvm/topi/python')
sys.path.append('/tvm/nnvm/python')
sys.path.append('/tvm/vta/python')

In [16]:
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops_fpga import make_conv1d_fpga_scheduler
from src.ops_fpga import make_conv1d_fpga_function

M = 16384
N = 32
dtype = 'float32'
a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
b_np = np.convolve(a_np, w_np)

info = make_conv1d_fpga_scheduler(M, N)
f = make_conv1d_fpga_function(info)
b_out = f(a_np, w_np)

print("Answer:", b_np)
print("Output:", b)

OSError: libcudnn.so.7: cannot open shared object file: No such file or directory

In [20]:
import vta
s = info['s']
A = info['input_A']
W = info['input_B']
O = info['output_C']
print(vta.lower(s, [A, W, O], simple_mode=True))

OSError: libLLVM-6.0.so.1: cannot open shared object file: No such file or directory

In [ ]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_1dconv_fpga.py